In [ ]:
import os, glob
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import wrftools as wrf
# import importlib
# importlib.reload(wrf)

In [ ]:
def get_Pbar(Pr,r): #exact dicrete mean P (within radius r)
    Par0=Pr[0]*r[0]**2
    Par=Pr[1:]*(r[1:]**2-r[:-1]**2)
    return(np.cumsum(np.insert(Par,0,Par0))/(r**2))
    
def get_row(fname,run):
    V10max,rV10max,zmax,t=wrf.wrf2max(run,fname,'V')
    vt10max,rvt10max,zmax,t=wrf.wrf2max(run,fname,'vt')
    Pmin,rPmin,Pzmin,t=wrf.wrf2max(run,fname,'P',minim=True)
    Pr=wrf.getWRF(wrf.wopath(run,fname),'P','az',force=False)
    Vt=wrf.getWRF(wrf.wopath(run,fname),'vt','az',force=False)
    r=wrf.getRcoord(wrf.wopath(run,fname))
    dr=r[1]-r[0]
    Pbar=get_Pbar(Pr,r)
    row = pd.Series({
        't':t,'vt10max':vt10max,'rvt10max':rvt10max,'V10max':V10max,'rV10max':rV10max,\
        'pmin':Pmin,'Pr':Pr,'Pbar':Pbar,'r':r,'Vt10':Vt
    })
    return(row)
    
def get_run_max_df(flist,run):
    df = pd.DataFrame(columns=['t','vt10max','rvt10max','V10max','rV10max','pmin','Pr','Pbar','r','Vt10'])
    for fname in flist:
        print(fname)
        df=df.append(get_row(fname,run),ignore_index=True)
    return(df)

In [ ]:
dpath='../run'
fname='wrfout_d03_*'
tlist=['03','04','05','06','07','08','09','10','11','12','13']
runlist=['run_land_dry_t' + t + '' for t in tlist]
# runlist=['run_CTRL_rr']
runlist

for run in runlist:
    wolist=wrf.wolist(run)[:24*15]
    df=get_run_max_df(wolist,run)
    sname='rundf/df_' + run + '.pkl'
    print(df)
    df.to_pickle(sname,protocol=4)